In [11]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install openrouteservice

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [105]:
import requests
from geopy.geocoders import Nominatim
import openrouteservice


In [14]:
address = "Calle Serrano 60,28001, Madrid"
ORS_API_KEY = '5b3ce3597851110001cf6248bc3b029ec4b443c7bca64b6d21cd1538'


In [ ]:

pip install opencage

In [80]:
import pandas as pd
import requests
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic

# Function to get coordinates using OpenCage Geocoder
def get_coordinates(address, api_key):
    geocoder = OpenCageGeocode(api_key)
    result = geocoder.geocode(address)
    if result and len(result):
        return result[0]['geometry']['lat'], result[0]['geometry']['lng']
    else:
        raise ValueError("Address not found.")

# Function to calculate the distance between two coordinates
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Function to find the nearest train station using OpenStreetMap Overpass API
def get_nearest_train_station(lat, lon):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node
      ["public_transport"="platform"]["bus"="yes"]
      (around:5000,{lat},{lon});
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    if not data['elements']:
        raise ValueError("No public transport found within 5km radius.")

    # Find the nearest train station
    nearest_station = None
    min_distance = float('inf')
    for element in data['elements']:
        station_coords = (element['lat'], element['lon'])
        distance = geodesic((lat, lon), station_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = element

    return nearest_station, min_distance

# Function to calculate all distances for a given address
def calculate_distances(address, api_key):
    # Predefined coordinates
    city_center = (40.416775, -3.703790)  # Puerta del Sol, Madrid
    castellana = (40.448466, -3.690612)  # Castellana, Madrid

    try:
        # Get coordinates for the given address
        lat, lon = get_coordinates(address, api_key)
        address_coords = (lat, lon)

        # Calculate distances
        distance_to_city_center = calculate_distance(address_coords, city_center)
        distance_to_castellana = calculate_distance(address_coords, castellana)

        # Find the nearest train station
        _, distance_to_station = get_nearest_train_station(lat, lon)

        return distance_to_city_center, distance_to_castellana, distance_to_station
    except ValueError as e:
        print(f"Error for address '{address}': {e}")
        return None, None, None

# Function to apply to DataFrame
def add_distance_columns(df, address_column, api_key):
    distances = df[address_column].apply(lambda x: calculate_distances(x, api_key))
    df[['Distance to City Center', 'Distance to Castellana', 'Distance to Nearest Public']] = pd.DataFrame(distances.tolist(), index=df.index)
    return df

# Example usage
#address = "Calle Serrano 60, 28001, Madrid"
#api_key = "YOUR_OPENCAGE_API_KEY"  # Replace with your OpenCage API key




In [89]:
# Load the dataset
df = pd.read_csv('scraped_data_cleaned_Emanuela_22Jul24.csv')
#df = df[df['m2']<150]
#df = df[[column for column in df.columns if column not in ['price_m2_Q4_2023', 'price_m2_Q3_2023', 'price_m2_Q1_2024']]]

#df_unseen = df.iloc[:1000]

#df = df.drop(df_unseen.index)

df.shape

(127947, 26)

In [87]:
df.head(2)

,price,has_orientation_south,has_orientation_north,has_orientation_west,has_orientation_east,rooms,bathrooms,has_terrace,m2,elevator,...,built_year,has_pool,has_ac,distance_city_center,distance_metro,distance_castellana,price_m2_Q2_2024,district_affordability,CAGR,airbnb_price_per_night
1294,345000.0,1.0,0.0,0.0,0.0,1,1.0,0.0,64.0,1,...,2003.0,1.0,1.0,NaN,NaN,NaN,3993,45.5,4.8,100.125000
1295,1215000.0,0.0,0.0,0.0,1.0,2,2.0,0.0,81.0,1,...,NaN,0.0,1.0,NaN,NaN,NaN,6062,48.4,5.9,265.809524


In [88]:
df.columns

Index(['price', 'has_orientation_south', 'has_orientation_north',
       'has_orientation_west', 'has_orientation_east', 'rooms', 'bathrooms',
       'has_terrace', 'm2', 'elevator', 'garage', 'neighbourhood', 'district',
       'built_year', 'has_pool', 'has_ac', 'distance_city_center',
       'distance_metro', 'distance_castellana', 'price_m2_Q2_2024',
       'district_affordability', 'CAGR', 'airbnb_price_per_night'],
      dtype='object')

In [ ]:
# Add distance columns
api_key = "46f6ec6a67a04a279a29e10a9ae7b28a"  # Replace with your OpenCage API key
df = add_distance_columns(df, 'address', api_key)

In [83]:
df

,address,Distance to City Center,Distance to Castellana,Distance to Nearest Public
0,"Calle Serrano 60, 28001, Madrid",1.960056,2.206531,0.091622
1,"Gran Via, 28013, Madrid",0.526635,3.360576,0.054283


In [101]:
import requests
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic

# Hardcoded OpenCage API key
API_KEY = "46f6ec6a67a04a279a29e10a9ae7b28a"  

# Function to get coordinates using OpenCage Geocoder
def get_coordinates(address, api_key):
    geocoder = OpenCageGeocode(api_key)
    result = geocoder.geocode(address)
    if result and len(result):
        return result[0]['geometry']['lat'], result[0]['geometry']['lng']
    else:
        raise ValueError("Address not found.")

# Function to calculate the distance between two coordinates
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Function to find the nearest tram station using OpenStreetMap Overpass API
def get_nearest_tram_station(lat, lon):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node
      ["public_transport"="platform"]["bus"="yes"]
      (around:5000,{lat},{lon});
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    if not data['elements']:
        raise ValueError("No tram stations found within 5km radius.")

    # Find the nearest tram station
    nearest_station = None
    min_distance = float('inf')
    for element in data['elements']:
        station_coords = (element['lat'], element['lon'])
        distance = geodesic((lat, lon), station_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = element

    return nearest_station, min_distance

def calculate_distances(address):#,street, postal_code, city):
#def calculate_distances(street, postal_code, city):
    #address = f"{street}, {postal_code}, {city}"
    
    # Predefined coordinates
    city_center = (40.416775, -3.703790)  # Puerta del Sol, Madrid
    castellana = (40.448466, -3.690612)  # Castellana, Madrid

    try:
        # Get coordinates for the given address
        lat, lon = get_coordinates(address, API_KEY)
        address_coords = (lat, lon)

        # Calculate distances
        distance_to_city_center = calculate_distance(address_coords, city_center)
        distance_to_castellana = calculate_distance(address_coords, castellana)

        # Find the nearest tram station
        _, distance_to_station = get_nearest_tram_station(lat, lon)

        return {
            "distance_city_center": distance_to_city_center,
            "distance_castellana": distance_to_castellana,
            "distance_metro": distance_to_station
        }

    except ValueError as e:
        print(f"Error for address '{address}': {e}")
        return None




In [102]:
# Example usage

address = "Calle Serrano 60,28001, Madrid"
#street = "Calle Serrano 60"
#postal_code = "28001"
#city = "Madrid"
#distances = calculate_distances(street, postal_code, city)
distances = calculate_distances(address)
if distances:
    for key, value in distances.items():
        print(f"{key}: {value}")

distance_city_center: 1.9600564527428441
distance_castellana: 2.2065305089437808
distance_metro: 0.09162243556217249


In [103]:
street = "Calle Serrano 60"
postal_code = "28001"
city = "Madrid"
address = f"{street}, {postal_code}, {city}"
address

'Calle Serrano 60, 28001, Madrid'

In [109]:
from opencage.geocoder import OpenCageGeocode

In [111]:
result = OpenCageGeocode.geocode(address)
result


TypeError: OpenCageGeocode.geocode() missing 1 required positional argument: 'query'

In [139]:
from opencage.geocoder import OpenCageGeocode

# OpenCage API key
API_KEY = '46f6ec6a67a04a279a29e10a9ae7b28a'  # Replace with your actual OpenCage API key

def get_district_and_neighborhood(address, api_key):
    geocoder = OpenCageGeocode(api_key)
    result = geocoder.geocode(address)
    
    if result and len(result):
        components = result[0]['components']
        neighborhood = components.get('quarter')
        district = components.get('suburb')
        print(result)
        return district, neighborhood
    return None, None

# Example address
street = "C. de Noblejas"
postal_code = "28013"
city = "Madrid"
address = f"{street}, {postal_code}, {city}"

# Get district and neighborhood
district, neighborhood = get_district_and_neighborhood(address, API_KEY)

# Print district and neighborhood
if district and neighborhood:
    print(f"District: {district}, Neighborhood: {neighborhood}")
else:
    print("District or Neighborhood not found.")


[{'annotations': {'DMS': {'lat': "40° 25' 0.60348'' N", 'lng': "3° 42' 42.99480'' W"}, 'MGRS': '30TVK3959974266', 'Maidenhead': 'IN80dk40na', 'Mercator': {'x': -413211.605, 'y': 4898992.911}, 'NUTS': {'NUTS0': {'code': 'ES'}, 'NUTS1': {'code': 'ES3'}, 'NUTS2': {'code': 'ES30'}, 'NUTS3': {'code': 'ES300'}}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?way=7988358#map=17/40.41683/-3.71194', 'note_url': 'https://www.openstreetmap.org/note/new#map=17/40.41683/-3.71194&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=40.41683&mlon=-3.71194#map=17/40.41683/-3.71194'}, 'UN_M49': {'regions': {'ES': '724', 'EUROPE': '150', 'SOUTHERN_EUROPE': '039', 'WORLD': '001'}, 'statistical_groupings': ['MEDC']}, 'callingcode': 34, 'currency': {'alternate_symbols': [], 'decimal_mark': ',', 'html_entity': '€', 'iso_code': 'EUR', 'iso_numeric': '978', 'name': 'Euro', 'smallest_denomination': 1, 'subunit': 'Cent', 'subunit_to_unit': 100, 'symbol': '€', 'symbol_first': 0, 'thousands_separator':

In [141]:
round(646700, -3)

647000